# EDA of Bundesliga Football Data

In this project, I've tried to make an exploratory data analysis and visualized some key points regarding Bundesliga metrics from the dataset in this link: https://www.kaggle.com/slehkyi/extended-football-stats-for-european-leagues-xg

Note: This dataset only includes domestic league stats, therefore Champions League, Europa League or DFB Pokal performances weren't put into consideration in this project.

In [ ]:
#Necessary library importations and creating the dataframe
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

df = pd.read_csv("../input/extended-football-stats-for-european-leagues-xg/understat_per_game.csv")


In [ ]:
#Getting a general view of the data
print(df.head())
df.date = pd.to_datetime(df.date)
df.describe()
df.info()

In [ ]:
#Definitely not necessary, although I just couldn't bear to see it wrote like "Fortuna Duesseldorf" so 
#fixed that
df["team"] = df.team.replace("Fortuna Duesseldorf" ,"Fortuna Düsseldorf")

#Simple function to extract a specific year from the year column
def season_date(frame, year):
    new = frame[frame["year"] == year]
    return new
df_2019 = season_date(df, 2019)
bundesliga = df_2019[df_2019["league"] == "Bundesliga"]
df_2019.head()

#  Definition of some metrics according to the original author of the dataset

xG - expected goals metric, it is a statistical measure of the quality of chances created and conceded(from understat.com)

xG_diff - difference between actual goals scored and expected goals.

npxG - expected goals without penalties and own goals.

xGA - expected goals against.

xGA_diff - difference between actual goals missed and expected goals against.

npxGA - expected goals against without penalties and own goals.

npxGD - difference between "for" and "against" expected goals without penalties and own goals.

ppda_coef - passes allowed per defensive action in the opposition half (power of pressure)

oppda_coef - opponent passes allowed per defensive action in the opposition half (power of opponent's pressure)

deep - passes completed within an estimated 20 yards of goal (crosses excluded)

deep_allowed - opponent passes completed within an estimated 20 yards of goal (crosses excluded)

xpts - expected points

xpts_diff - difference between actual and expected points

In [ ]:
#Plot the cumulative change in total points and therefore the rankings in time

ax = plt.subplots(figsize=(15, 10))
sns.set_context("notebook")
sns.set_style("dark")
sns.lineplot(bundesliga.date, bundesliga.groupby("team").pts.cumsum(), hue="team", data=bundesliga, palette = "Dark2")
sns.despine(top=True, bottom=True, right=True)
plt.title("Change in Bundesliga rankings by time")
plt.xlabel("Date")
plt.ylabel("Points total")

In [ ]:
#Grouped by the team column, then took the means of xG and scored columns 
#in order to compare average xG and goal amounts by team
bundesliga_expected_actual = df_2019[df_2019["league"] == "Bundesliga"].groupby("team")[["xG", "scored"]].mean()
#Reset the index for easier column operations
bundesliga_expected_actual = bundesliga_expected_actual.reset_index()
#Sort values by goal expectation
bundesliga_expected_actual = bundesliga_expected_actual.sort_values("xG", ascending = False)
display(bundesliga_expected_actual)

In [ ]:
#Again group by the team columns, although this time compare with
#average successful deep passes per game
bundesliga_deep = bundesliga.groupby("team").deep.mean()
bundesliga_deep = bundesliga_deep.reset_index()
bundesliga_deep = bundesliga_deep.sort_values("deep", ascending = False)
display(bundesliga_deep)

In [ ]:
#Same as the above two cells
bundesliga_points = df_2019[df_2019["league"] == "Bundesliga"].groupby("team")[["pts", "xpts"]].sum()
bundesliga_points = bundesliga_points.reset_index()
bundesliga_points = bundesliga_points.sort_values("pts", ascending = False)
bundesliga_xpoints = bundesliga_points.sort_values("xpts", ascending = False)
display(bundesliga_points)

In [ ]:
#Create a column named press_eff showing the ratio of passes that teams allowed 
#their opponents to make inside their box before recieving the ball

bundesliga["press_eff"] = bundesliga["deep_allowed"]/bundesliga["ppda_coef"]
bundesliga_press = bundesliga.groupby("team").press_eff.mean()
bundesliga_press = bundesliga_press.reset_index()
bundesliga_press = bundesliga_press.sort_values("press_eff", ascending = False)
display(bundesliga_press)

In [ ]:
#Create a column named peneteration showing the ratio of passes that teams made 
#inside opposition box before their opponents recovered the ball

bundesliga["peneteration"] = bundesliga["deep"]/bundesliga["oppda_coef"]
bundesliga_penet = bundesliga.groupby("team").peneteration.mean()
bundesliga_penet = bundesliga_penet.reset_index()
bundesliga_penet = bundesliga_penet.sort_values(by = "peneteration", ascending = False)
display(bundesliga_penet)

In [ ]:
#Plot bar graphs of average xpts and deep for each team side-by-side

sns.set_context("talk")
sns.set_style("darkgrid")

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
plt.subplots_adjust(wspace = 0.5)
sns.barplot(x="xG", y="team", palette="rocket_r", data=bundesliga_expected_actual, ax=ax1)
ax1.set(title=("Average Goal Expectations per Team"), xlabel=("Goal Expectation"), ylabel=("Teams"))

sns.barplot(x="deep", y="team", palette="rocket_r", data=bundesliga_deep, ax=ax2)
plt.title("Average Successful Deep Passes per Game")
plt.xlabel("Successful deep (within 20 yards of goal) passes per game")
plt.ylabel("Teams")
plt.show()

# Comparison of deep passing and goal expectations (xG) by team 
We can see on the above left graph that Bayern München had the highest average xG per game, while Fortuna Düsseldorf was last place. 

The metric "deep", which I also used for the right plot was defined by the original author of this dataset as:

"deep - passes completed within an estimated 20 yards of goal (crosses excluded)"

According to the graph, again, Bayern performed best regarding the metric "deep", which pretty much stands for passes around and inside the opponent's penalty box. This might offer a general idea on how teams operate in terms of offensive play, as an example Bayern completed a little bit more than three times the amount of passes Paderborn could (which are last place), and we could extract insights such as: "Bayern have internalized a mentality that mostly employs playing into box with organized passing", 

or: "Paderborn couldn't complete many passes around the box, therefore it indicates that they emphasize a more direct approach when attacking or cross more often" 

using this metric alone.

In [ ]:
#Plot bar graphs of total pts and xpts for each team side-by-side

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
plt.subplots_adjust(wspace = 0.7)
sns.barplot(x="pts", y="team", palette="rocket_r", data=bundesliga_points, ax=ax1)
ax1.set(title = ("Total Points Collected by Teams"), xlabel=("Total Points"), ylabel=("Teams"), xlim=[0, 85])

sns.barplot(x="xpts", y="team", palette="rocket_r", data=bundesliga_xpoints, ax=ax2)
plt.xlabel("Expected Points Total")
plt.ylabel("Teams")
plt.title("Total Points Expected to Have Been Collected by Teams")
plt.xlim([0, 85])
plt.show()

# Comparison of actual and expected points collected by team in Bundesliga 19/20 season
These couple of plots are more interesting to look at in my opinion mostly since they're metrics in direct correlation with each other. 

As could be seen above, in terms of actual points collected Bayern sit on the top, then come Dortmund and Leipzig. Although on the right-side bar plot, that order changes by Bayern, Leipzig, and Gladbach which have the same expected points as Dortmund. This means that Dortmund overperformed in terms of actual count of total points collected, while Leipzig and Gladbach underperformed. In fact, Bayern also underperformed since they went above 80 points in reality while the expected points metric for them is below that number.

Let's create a ratio of actual points divided by expected points and visualize that for each team, so that we could see much easier which teams underperformed/overperformed and to what extent.

In [ ]:
#Create variables that represent overperformance and underperformance
#in terms of collecting points
high_xpts = bundesliga_points[bundesliga_points["xpts"] >= bundesliga_points["pts"]]
high_pts = bundesliga_points[bundesliga_points["pts"] > bundesliga_points["xpts"]]
#Create a new column on the dataframe that was grouped by teams and expected points
#that show the ratio of actual points to expected points
bundesliga_points["pts/xpts"] = bundesliga_points["pts"] / bundesliga_points["xpts"]

In [ ]:
#Plot bar graphs of average pts divided by xpts for each team to understand
#how much each team lived up to expectations in terms of point collection

ax = plt.subplots(figsize=(15, 10))
sns.barplot("pts/xpts", "team", data=bundesliga_points.sort_values(by="pts/xpts", ascending=False), palette="GnBu")
plt.title("Performance in point collection")
plt.xlabel("Points to expected points ratio")
plt.ylabel("Team")
plt.show()

# Evaluation of whether if the teams could live up to expectations in terms of points collected
So, let's try to get a more intuitive view of this bar plot. Numbers on the x-axis represent the ratio that we talked about on the above cell: actual points/expected points. Setting this as our starting point, a ratio on 1.0 would mean that the actual amount of points gathered by a team were 100% consistent with the expectation metric. So, simply put, it could be said that teams that are above the ratio 1.0 *overperformed*, while those under 1.0 *underperformed*.

Let's try the same thing with the actual amount of goals scored and goal expectations!

In [ ]:
#Create variables that represent overperformance and underperformance
#in terms of goal scoring
high_xg = bundesliga_expected_actual[bundesliga_expected_actual["xG"] >= bundesliga_expected_actual["scored"]]
high_scoring = bundesliga_expected_actual[bundesliga_expected_actual["scored"] > bundesliga_expected_actual["xG"]]
#Create a new column on the dataframe that was grouped by teams and goal expectations
#that show the ratio of goals to xG
bundesliga_expected_actual["goal/xG"] = bundesliga_expected_actual["scored"] / bundesliga_expected_actual["xG"]

In [ ]:
#Check if all's well
bundesliga_expected_actual.head()

In [ ]:
#Plot bar graphs of average goals and expected for each team to see
#how well they performed

ax = plt.subplots(figsize=(15, 10))
sns.barplot("goal/xG", "team", data=bundesliga_expected_actual.sort_values(by="goal/xG", ascending=False), palette="RdYlBu")
plt.ylabel("Team")
plt.xlabel("Goal to Goal Expectation Ratio")
plt.title("Goal/xG Ratios of Bundesliga Clubs in 19/20 Season")

# Evaluation of whether if the teams could live up to expectations in terms of goals scored
This bar plot essentially gives us a visual represantation of the teams' "goal efficiency" rates, where 1.0 means similarly to the bar plot above that the team in question scored as many goals as they were expected to. 

Therefore the teams with a higher ratio, such as Borussia Dortmund or Werder Bremen, could be said to have overperformed in terms of converting chances to goals compared to those that are located at the lower sections of the graph, like Wolfsburg or Borussia Mönchengladbach.

Instead of the league in general, let's focus on a single team now. I chose RB Leipzig, since I've been enjoying their playing style and Julian Nagelsmann's display of his managerial skill set.

In [ ]:
#Plot a bar graph to show how many passes were allowed 
#before recovery inside a team's penalty box

ax = plt.subplots(figsize=(15, 10))
sns.barplot("press_eff", "team", data=bundesliga_press, palette="RdYlBu")
plt.xlabel("Allowed opponent deep passes")
plt.ylabel("Team")
plt.title("Passes made inside teams' penalty box until recovery")

In [ ]:
#Plot a bar graph to show how many passes teams allowed 
#inside their penalty box before recovering the ball 

ax = plt.subplots(figsize=(15, 10))
sns.barplot("peneteration", "team", data=bundesliga_penet, palette="RdYlBu")
plt.xlabel("Passes made inside oppponent's box before loss of possession")
plt.ylabel("Team")
plt.title("Passes made inside opponents' penalty box until loss")

This chart provides insight on how often teams allow their opponents to approach their penalty box by passing. Reading from the bar plot above, Borussia Dortmund were the best at disabling teams from working the ball inside their penalty box. And to counter that, Leipzig appear to be league leaders at penetrating defenses to enter opposition box with the ball and pass the most before losing possession.

These two plots above show how likely teams are to fill up open spaces and passing lanes inside their own penalty boxes, and how good they are at feeding off from the ones the opponents leave.

In [ ]:
#Create scatter plots side-by-side showing how goal expectation changes
#by metrics ppda_coef, oppda_coef, deep and deep_allowed

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
plt.subplots_adjust(wspace = 0.3)
sns.scatterplot(x="ppda_coef", y="xG", data=bundesliga, ax=ax1, alpha=0.6)
sns.scatterplot(x="oppda_coef", y="xG", data=bundesliga, ax=ax1, alpha=0.6)
ax1.set(title = ("Change of xG by OPPDA/PPDA"), xlabel=("OPPDA/PPDA"), ylabel=("xG"))
ax1.legend(["PPDA","OPPDA"])

sns.scatterplot(x="deep", y="xG", data=bundesliga, ax=ax2, alpha=0.6)
sns.scatterplot(x="deep_allowed", y="xG", data=bundesliga, ax=ax2, alpha=0.6)
plt.xlabel("Deep passes made/allowed")
plt.ylabel("xG")
plt.title("Change of xG by deep passes made/allowed")
plt.legend(["Deep Made", "Deep Allowed"])
plt.show()

There doesn't appear to be a pattern between on how teams' xG change depending on PPDA/OPPDA or deep passes made/allowed metrics.

In [ ]:
#Create scatter plots side-by-side showing how xpts changes
#by metrics ppda_coef, oppda_coef, deep and deep_allowed

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
plt.subplots_adjust(wspace = 0.3)
sns.scatterplot(x="ppda_coef", y="xpts", data=bundesliga, ax=ax1, alpha=0.6)
sns.scatterplot(x="oppda_coef", y="xpts", data=bundesliga, ax=ax1, alpha=0.6)
ax1.set(title = ("Change of expected points by OPPDA/PPDA"), xlabel=("OPPDA/PPDA"), ylabel=("Expected Points"))
ax1.legend(["PPDA","OPPDA"])

sns.scatterplot(x="deep", y="xpts", data=bundesliga, ax=ax2, alpha=0.6)
sns.scatterplot(x="deep_allowed", y="xpts", data=bundesliga, ax=ax2, alpha=0.6)
plt.xlabel("Deep passes made/allowed")
plt.ylabel("Expected Points")
plt.title("Change of expected points by deep passes made/allowed")
plt.legend(["Deep Made", "Deep Allowed"])
plt.show()

And that trend of no relation seems to follow for expected points aswell. Therefore it would be safe to say that deep passing or PPDA/OPPDA metrics have no direct effect on the results teams have received.

In [ ]:
#Filter the Bundesliga dataset to get rows that only contain data for RB Leipzig
leipzig = bundesliga[bundesliga["team"] == "RasenBallsport Leipzig"]
leipzig.head()

In [ ]:
#Create a heatmap to visulize the Pearson correlation coefficient for RB Leipzig metrics
sns.set_context("talk")
ax=plt.subplots(figsize=(10,10))
sns.heatmap(leipzig.corr())
plt.title("RB Leipzig's 19/20 season correlation matrix")
plt.savefig("leipzig_matrix")

# Correlation between RB Leipzig data features 
The correlation matrix is a very intuitive visualization method to discover *linear* relationships between features. Although it must be noted that a low Pearson coefficient doesn't necessarily indicate that the features in question are not correlated at all, this value only informs us of whether if the selected features are *linearly* correlated. Therefore, a very strong polynomial correlation (such as a parabola) between two variables will still return a low Pearson coefficient.

In [ ]:
#Create variables by filtering which venue the game was played at
leipzig_home = leipzig[leipzig["h_a"] == "h"]
leipzig_away = leipzig[leipzig["h_a"] == "a"]

In [ ]:
#Calculate the average goal expectations by venue 
print(leipzig_home.xG.mean(), leipzig_away.xG.mean())

In [ ]:
#See how many games RB Leipzig won, lost or drew
leipzig["result"].value_counts()

In [ ]:
#Create a bar plot showing the difference between how much difference
#there was between goals that were actually scored and should've been
#scored in average by each result

leipzig_r_xg = leipzig.groupby("result").xG_diff.mean().head()
leipzig_r_xg = leipzig_r_xg.reset_index()

sns.set_style("darkgrid")
sns.set_context("poster")
ax = plt.subplots(figsize=(8, 8))
sns.barplot("result", "xG_diff", data=leipzig_r_xg, palette="cividis")
plt.title("Difference of Actual Goals - Expected Goals")
plt.ylabel("Difference")
plt.xlabel("Result")
plt.show()

# Goals scored - goal expectation difference per result category
Here's another plot that I find enjoyable to look at. Each bar stands for a result (win/draw/lose) and the y-axis represents the mean difference between the actual count of goals scored and the goal expectation metric. Or to mathematically notate:

(The average of actual goal counts for each result category) - (The average of xG for each result category)

According to the above graph, Leipzig have exceeded expectations in terms of goal scoring for the games they won, which means that they scored fewer goals than they were supposed to. And interestingly enough, Leipzig apparently have overperformed in the games they couldn't win and scored more goals than they were expected to. This might indicate that Leipzig won the games they bagged 3 points by dominating their opponents, whereas for the ones they couldn't win the team were outperformed on the pitch and couldn't get many scoring opportunities, yet somehow managed to make the ball cross the goal line.

In [ ]:
plt.clf()

In [ ]:
#Plot two bar graphs side-by-side, one showing how the average amounts of goals
#scored per game and the other xG per game depending on the venue played

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

sns.set_context("poster")

sns.barplot("h_a", "scored", data=leipzig, ax=ax1, ci=False, palette = "cividis")
ax1.set(xlabel=("Home/Away"), ylabel=("Goals scored"), ylim=[0, 2.8])

sns.barplot("h_a", "xG", data=leipzig, ax=ax2 , ci=False, palette = "cividis")
plt.ylim([0, 2.8])
plt.xlabel("Home/Away")
plt.ylabel("Goal Expectation")
plt.show()

# Comparison of goals scored and xG by venue played (home/away)
The plots above allow us to compare the average counts of goals scored and expected per game by whether the game was played at Red Bull Arena or not. 

It appears that Leipzig have performed better at away games in terms of scoring goals and there's just a slight difference between the actual and expected amount of goals for away games. 

For home games however, the difference between actual goals and xG seem to differ roughly by 0.5 goals in average. Basically, Leipzig have (again, roughly) scored a goal more than they should have for each two game played at home. This might've been caused by a variety of factors, such as fan support or opponents shifting to a more reactive style of play since it's an away game. Therefore it could lead to incorrect assumptions if we were to draw conclusions from these graphs alone.

In [ ]:
#Create a count plot that visualizes the amounts of wins, draws and losses
#depending on the venue played

results_h_a = leipzig.groupby("h_a").result.value_counts()
ax = plt.subplots(figsize = (8, 8))

sns.countplot("result",data=leipzig,hue="h_a", palette = "cividis")
plt.yticks([0, 2, 5, 8, 10])
plt.title("Leipzig 19/20 game result counts by venue played")
plt.xlabel("Result")
plt.ylabel(" ")
plt.legend(["Away", "Home"])
plt.show()
print(leipzig.groupby("h_a").result.value_counts())

# Result counts by venue the game was played
Just like the bar plots under the above cells which depicted that Leipzig had scored more goals away, the team appears to have also won almost 50% more games away compared to home matches. The significance of their away performance manifests itself further considered that Leipzig has more draws at home than wins. 

It wouldn't be correct to draw any conclusions without further analysis although, on whether if they just overperformed at away games or their home record's an outcome of underperformance. These statements might sound like they are essentially addressing the same thing at first, however it could be seen that is not the case after going over them once more carefully.

# What does it take to win the title?

We looked into RB Leipzig's performance data, and we saw that RB Leipzig should've been second place if Borussia Dortmund didn't overperform in terms of point collected throughout the season, which'd put them right under the reigning champions, Bayern München.

Let's compare Leipzig and Bayern to see what differences their seasons had in terms of numbers and try to make sense of it.

In [ ]:
#Select rows with only Bayern and Leipzig as teams
leipzig_bayern = bundesliga[(bundesliga["team"] == "Bayern Munich") | (bundesliga["team"] == "RasenBallsport Leipzig")]
leipzig_bayern.head()

In [ ]:
#Compare the results of Bayern and Leipzig
ax = plt.subplots(figsize = (8, 8))
sns.set_context("notebook")
sns.countplot("result", hue="team", data=leipzig_bayern, palette="RdBu")
plt.title("Result Counts of Bayern Munich vs RB Leipzig")
plt.xlabel("Result")
plt.ylabel(" ")

In [ ]:
#Create a scatter plot visualizing how each team performed regarding deep passes
#under different levels of pressure received 

ax = plt.subplots(figsize = (10, 10))
sns.set_context("talk")
sns.scatterplot("oppda_coef", "deep", hue="team", data=leipzig_bayern, palette="RdBu")
plt.title("Change of passes around opposition box by opponent's pressing strength")
plt.ylabel("Successful passes")
plt.xlabel("Opposition pressing strength")
print(leipzig_bayern.groupby("team").deep.mean())
print(leipzig_bayern.groupby("team").oppda_coef.mean())

What this graph and the mean values of oppda_coef and deep metrics tell us is that Leipzig received more pressure (50% tougher) from their opponents, or simply wasn't good enough at passing the ball around under pressure as much as Bayern. 

This might be an indicator of that RB Leipzig, compared to Bayern München, had more troubles peneterating through defenses that sit back and focus on leaving no empty spaces.

In [ ]:
#Plot a bar graph showing how many passes were made in average before losing the ball 
#inside the opposition box

ax = plt.subplots(figsize=(8, 8))
sns.barplot("team", bundesliga_penet["peneteration"], data=bundesliga_penet[(bundesliga_penet["team"] == "RasenBallsport Leipzig") | 
                                                       (bundesliga_penet["team"] == "Bayern Munich")], palette="RdYlBu")
plt.xlabel("Passes inside oppponent's box before loss of possession")
plt.ylabel("Team")
plt.title("Passes made inside opponents' penalty box until loss")
plt.ylim([0, 0.85])

In average, roughly for each 0.7 pass made inside the opponent's box, Leipzig had their ball intercepted while that number is closer to 0.6 for Bayern. Therefore it could be said that Leipzig were more potent when it came to working the ball inside the box.

In [ ]:
#Create a scatter plot that visualizes how many passes teams allowed inside
#their own box before recovering the ball

ax = plt.subplots(figsize = (10, 10))
sns.set_context("talk")
sns.scatterplot("ppda_coef", "deep_allowed", hue="team", data=leipzig_bayern, palette="RdBu")
plt.title("Passes inside box vs. pressing strength")
plt.ylabel("Opposition successful passes")
plt.xlabel("Pressing strength")
print(leipzig_bayern.groupby("team").deep_allowed.mean())
print(leipzig_bayern.groupby("team").ppda_coef.mean())

Leipzig seem to have allowed their opponents pass inside their box 30% more in average compared to Bayern, and again, they let their opponents pass 25% more than Bayern did before intercepting the ball.

In [ ]:
#Plot a bar graph showing how many of the passes were made inside
#Bayern and Leipzig's box before recovery

ax = plt.subplots(figsize=(8, 8))
sns.barplot("team", "press_eff", data=bundesliga_press[(bundesliga_press["team"] == "RasenBallsport Leipzig") | 
                                                       (bundesliga_press["team"] == "Bayern Munich")], palette="RdYlBu_r")
plt.xlabel("Allowed opponent deep passes")
plt.ylabel("Team")
plt.title("The percentage of passes made around teams' penalty box until recovery")

plt.ylim([0, 0.65])

This graph shows us that the two teams have shown similar behavior when it came to allowing their opponents pass inside their box. An average of each 0.45 of opposition passes were intercepted around the teams' penalty box. In other words, Bayern and Leipzig have recovered the ball for every 0.45 pass made inside their box.

Basically, both of these teams allowed their opponents to pass inside their box less than they could pass inside their opponents' box in average.